In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import KFold
import itertools
from sklearn.linear_model import LogisticRegression

# **LOADING DATASET**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
names = ['index','review', 'sentiment', 'label']
df = pd.read_csv('/content/drive/MyDrive/imdb/train_data.csv', sep=',', names=names, header=0)
#df_val = pd.read_csv('val.csv', sep=',', names=names, header=0)
#df=pd.concat((df_train, df_val))
df.dropna(how='any', inplace=True)
df.reset_index(drop=True, inplace=True)
df["review"] = df['review'].values.astype('U')
X = df['review'].to_numpy()
Y = df['label'].to_numpy()
print(X.shape)

(10000,)


In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.25, random_state=0)

# **VECTORIZING DATASET**

In [ ]:
MAX_FEATURES = 10000
cv = CountVectorizer(max_features = MAX_FEATURES)
cv.fit(X_tr)
X_train = cv.transform(X_tr)
X_train = X_train.todense()
X_tr=X_train


X_valid=cv.transform(X_val)
X_valid=X_valid.todense()
X_val=X_valid



# **REMOVING OUTLIERS**

In [ ]:
X_tr=np.array(X_tr)
Y_tr=np.array(Y_tr)

In [ ]:
clf_Iso = IsolationForest(random_state=np.random.RandomState(0),n_jobs = -1)
clf_Iso.fit(X_tr)
Y_Iso_Forest = clf_Iso.predict(X_tr)
result = np.where(Y_Iso_Forest == -1)
result = list(itertools.chain.from_iterable(result))

In [ ]:
X_removed = np.delete(X_tr,result,axis = 0)
if Y_tr is None:
    X_train=X_removed
else:
    Y_removed = np.delete(Y_tr,result,axis = 0)
X_tr=X_removed
Y_tr=Y_removed

In [ ]:
print(X_tr.shape, Y_tr.shape)

(7500, 10000) (7500,)


In [ ]:
num_samples = X_tr.shape[0]
num_features = X_tr.shape[1]
X_tr = np.reshape(np.array(X_tr), (num_samples, num_features))

In [ ]:
C = 1 
solver = 'sag' 
kf = KFold(n_splits=5)
logistic = LogisticRegression(max_iter=500, C=C, solver=solver)
acc_list = []
# Doing cross validation testing
for train_index, test_index in kf.split(X_tr):
    X_train, X_test = X_tr[train_index], X_tr[test_index]
    Y_train, Y_test = Y_tr[train_index], Y_tr[test_index]
    logistic.fit(X_train, Y_train)
    print("----Start Evaluating----")
    acc = logistic.score(X_test, Y_test)
    acc_list.append(acc)
    print("Testing Accuracy:", acc)
print("Mean testing accuracy:", sum(acc_list) / len(acc_list))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


----Start Evaluating----
Testing Accuracy: 0.8766666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


----Start Evaluating----
Testing Accuracy: 0.856


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


----Start Evaluating----
Testing Accuracy: 0.8613333333333333


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


----Start Evaluating----
Testing Accuracy: 0.8546666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


----Start Evaluating----
Testing Accuracy: 0.8533333333333334
Mean testing accuracy: 0.8604


In [ ]:
acc_val = logistic.score(X_val, Y_val)
print('Validation accuracy:', acc_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Validation accuracy: 0.8572
